In [2]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from pandas_datareader import data as web
import os 
import pickle
import numpy as np
from collections import Counter
from nsepy import get_history
from finta import TA
import copy
import math
from dateutil.relativedelta import relativedelta

style.use('ggplot')
%matplotlib qt
per_order_quantity = 10000

c:\users\asus\appdata\local\programs\python\python38-32\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
date_today = dt.date.today()
start_date = dt.datetime.strptime("2020-03-23" , '%Y-%m-%d').date() +  dt.timedelta(days=1)
with open("nifty500.pickle", "rb") as f:
    tickers = pickle.load(f)

2020-08-08 2020-03-24


In [74]:
Portfolio = {'Total_investment' : 0, 'Current_investment' : 0 , 'Money_generated' : 0, 'Total_trades' : 0, 'Successful_trades' : 0, 'Failed_trades': 0, 'Successful_shorts': 0, 'Failed_shorts': 0}
company_template = {'logs': [] , 'Total_investment': 0, 'Total_trades' : 0, 'P/L':0, 'Buy_price': 0, 'Buy_Quantity': 0 , 'Money_generated': 0, 'Successful_trades': 0, 'Failed_trades': 0, 'Successful_shorts': 0, 'Failed_shorts': 0}
Company_data = {}
for ticker in tickers:
    Company_data[ticker] = copy.deepcopy(company_template) 


In [75]:
def buy_hold_sell(df_curr, df_next):
    if df_curr['Remarks'] != None:
        if df_curr['Prev Close'] > df_curr['Close']:
            multiplier = 2
        else :
            multiplier  = 1
    if funds_per_trade/df_next['Open'] < 0:
        quantity = 1
    else:
        quantity = math.ceil(funds_per_trade/df_next['Open'])
    if df_curr['Score'] > 0:
        return "Buy", quantity*multiplier
    elif df_curr['Score'] < -2:
        return "Sell", quantity*multiplier 
    else: 
        return "Hold", 0


In [76]:
funds_per_trade = 10000

def test(ticker):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df = df[304:]
    for i in range(0, len(df)-1):
        bhs, quantity = buy_hold_sell(df.iloc[i], df.iloc[i+1])
        price =  df.iloc[i+1]['Open']
        if bhs == 'Buy':
            Company_data[ticker]['logs'].append([df.iloc[i]['Date'], bhs, price , quantity])            
            Company_data[ticker]['Total_investment'] += price*quantity 
            Company_data[ticker]['Total_trades'] += 1
            Company_data[ticker]['Buy_price'] = ( Company_data[ticker]['Buy_price']*Company_data[ticker]['Buy_Quantity'] + price*quantity) / (Company_data[ticker]['Buy_Quantity'] + quantity)
            Company_data[ticker]['Buy_Quantity'] += quantity
            Portfolio['Total_investment'] += price*quantity
            Portfolio['Current_investment'] += price*quantity
            Portfolio['Total_trades'] += 1
            print('Date :{}\tBuy\t{}\t{}'.format(df.iloc[i]['Date'], price, quantity))
        elif bhs == 'Sell':
            if Company_data[ticker]['Buy_Quantity'] != 0:
                Company_data[ticker]['logs'].append([df.iloc[i]['Date'], bhs, price , quantity])
                Company_data[ticker]['Total_trades'] += 1
                #print(Company_data[ticker]['Buy_price'], Company_data[ticker]['Buy_Quantity'])
                Company_data[ticker]['Money_generated'] += (Company_data[ticker]['Buy_Quantity']*price - Company_data[ticker]['Buy_price']*Company_data[ticker]['Buy_Quantity'])
                Portfolio['Money_generated'] += (Company_data[ticker]['Buy_Quantity']*price - Company_data[ticker]['Buy_price']*Company_data[ticker]['Buy_Quantity'])
                Portfolio['Total_trades'] += 1
                if (Company_data[ticker]['Buy_Quantity']*price - Company_data[ticker]['Buy_price']*Company_data[ticker]['Buy_Quantity'])> 0:
                    Company_data[ticker]['Successful_trades'] += 1
                    Portfolio['Successful_trades'] += 1
                else:
                    Company_data[ticker]['Failed_trades'] += 1
                    Portfolio['Failed_trades'] += 1
                Company_data[ticker]['P/L'] = (Company_data[ticker]['Money_generated']/Company_data[ticker]['Total_investment'] )*100
                Portfolio['Current_investment'] -= Company_data[ticker]['Buy_Quantity']*Company_data[ticker]['Buy_price']  
                print('Date :{}\tSell\t{}\t{}'.format(df.iloc[i]['Date'], price, Company_data[ticker]['Buy_Quantity']))
                print('\tCurrent_Profit : {}\t Total_Profit : {}'.format((Company_data[ticker]['Buy_Quantity']*price - Company_data[ticker]['Buy_price']*Company_data[ticker]['Buy_Quantity']), Company_data[ticker]['Money_generated']))
                Company_data[ticker]['Buy_Quantity'] = 0       
                Company_data[ticker]['Buy_price'] = 0             
            #SHORT TRADE 
            Company_data[ticker]['Money_generated'] += (price - df.iloc[i+1]['Close'])*quantity
            Portfolio['Money_generated'] += (price - df.iloc[i+1]['Close'])*quantity
            Company_data[ticker]['Total_trades'] += 1
            if (price - df.iloc[i+1]['Close'])*quantity > 0:
                Portfolio['Successful_shorts'] += 1
                Company_data[ticker]['Successful_shorts'] += 1
            else:
                Portfolio['Failed_shorts'] += 1
                Company_data[ticker]['Failed_shorts'] += 1   
            print('Date :{}\tShort\t{}\t{}'.format(df.iloc[i]['Date'], price, quantity))
            print('\tCurrent_Profit : {}\t Total_Profit : {}'.format((price - df.iloc[i+1]['Close'])*quantity, Company_data[ticker]['Money_generated']))
        else:
            print('Date :{}\tNo Transaction'.format(df.iloc[i]['Date']))
            

for ticker in tickers:
    test('HDFCBANK')
    display(Company_data['HDFCBANK'])
    break
    

Date :2020-03-25	Buy	870.1	12
Date :2020-03-26	Buy	944.95	11
Date :2020-03-27	Buy	880.0	12
Date :2020-03-30	Sell	853.8	35
	Current_Profit : -1512.6500000000015	 Total_Profit : -1512.6500000000015
Date :2020-03-30	Short	853.8	24
	Current_Profit : -194.40000000000055	 Total_Profit : -1707.050000000002
Date :2020-03-31	Short	863.85	12
	Current_Profit : 410.40000000000055	 Total_Profit : -1296.6500000000015
Date :2020-04-01	Short	843.0	24
	Current_Profit : 699.5999999999995	 Total_Profit : -597.050000000002
Date :2020-04-03	Short	874.0	24
	Current_Profit : -530.4000000000005	 Total_Profit : -1127.4500000000025
Date :2020-04-07	Buy	879.95	12
Date :2020-04-08	Buy	913.0	22
Date :2020-04-09	Buy	934.0	11
Date :2020-04-13	No Transaction
Date :2020-04-15	Sell	865.0	45
	Current_Profit : -1994.4000000000015	 Total_Profit : -3121.850000000004
Date :2020-04-15	Short	865.0	24
	Current_Profit : -354.0	 Total_Profit : -3475.850000000004
Date :2020-04-16	Short	915.0	11
	Current_Profit : 51.7000000000005	

{'logs': [['2020-03-25', 'Buy', 870.1, 12],
  ['2020-03-26', 'Buy', 944.95, 11],
  ['2020-03-27', 'Buy', 880.0, 12],
  ['2020-03-30', 'Sell', 853.8, 24],
  ['2020-04-07', 'Buy', 879.95, 12],
  ['2020-04-08', 'Buy', 913.0, 22],
  ['2020-04-09', 'Buy', 934.0, 11],
  ['2020-04-15', 'Sell', 865.0, 24],
  ['2020-04-17', 'Buy', 951.0, 11],
  ['2020-04-20', 'Buy', 925.0, 11],
  ['2020-04-21', 'Buy', 919.15, 22],
  ['2020-04-23', 'Buy', 933.0, 11],
  ['2020-04-24', 'Buy', 945.15, 22],
  ['2020-04-28', 'Sell', 935.0, 11],
  ['2020-04-29', 'Buy', 1001.4, 10],
  ['2020-04-30', 'Buy', 957.5, 11],
  ['2020-05-04', 'Sell', 938.0, 22],
  ['2020-05-06', 'Buy', 940.5, 11],
  ['2020-05-12', 'Sell', 951.0, 22],
  ['2020-05-13', 'Buy', 898.0, 12],
  ['2020-05-14', 'Sell', 890.5, 24],
  ['2020-05-27', 'Buy', 920.0, 11],
  ['2020-05-28', 'Buy', 944.0, 11],
  ['2020-05-29', 'Buy', 975.0, 11],
  ['2020-06-01', 'Buy', 986.7, 11],
  ['2020-06-03', 'Buy', 1028.05, 10],
  ['2020-06-05', 'Buy', 1060.0, 10],
  ['20

In [454]:
def bsh(Company_Result, price):
    heikenashi_changeover = Company_Result[' Change over hieken_ashi candle : (hand and leg shall be 1% of candle body (+4)']
    pos_score = Company_Result['Pos_Score'] - heikenashi_changeover
    neg_score = Company_Result['Neg_Score'] 
    pos_strat = Company_Result['Pos_Strat'] 
    neg_strat = Company_Result['Neg_Strat'] 
    buy_quantity = 1 + math.floor((per_order_quantity/price))
    if (neg_score == 0 and pos_score > 8) :
        return buy_quantity*2
    elif ((heikenashi_changeover == 4) and pos_score > neg_score*-1.5):
        return buy_quantity*2
    elif ((heikenashi_changeover == 4) and 1.5 * pos_score < -1*neg_score):
        return buy_quantity*-2
    elif (pos_score == 0 and neg_score <= -10):
        return buy_quantity*-2
    elif (neg_score == 0 and pos_score > 5) :
        return buy_quantity*1
    elif (pos_score == 0 and neg_score <= -5):
        return buy_quantity*-1
    else:
        return 0   
    
shorts = 0
for ticker in Results.index:
    if ticker not in portfolio['Stocks']:
        continue
    Company_Result = Results.loc[ticker]
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df.index = df.Date
    Open = df.iloc[day+2].Open
    High = df.iloc[day+2].High
    Low = df.iloc[day+2].Low
    Close = df.iloc[day+2].Close
    #print(Company_Result)    
    price = (High + Low)/2
    quantity = bsh(Company_Result,price)
    Company_Status = portfolio['Stocks'][ticker]

    
    if quantity == 0 :
        Company_Status['Current_Price'] = Close
        portfolio['Current_funds'] -= Company_Status['Current_Valuation']
        Company_Status['Current_Valuation'] = Close*Company_Status['Bought_Quantity']
        portfolio['Current_funds'] += Company_Status['Current_Valuation']
        if Company_Status['Investment'] != 0:
            Company_Status['Status'] = (Company_Status['Current_Valuation'] - Company_Status['Investment'])/Company_Status['Investment']
        
    if quantity > 0 :
        Company_Status['Bought_Price'] = ((Company_Status['Bought_Price']*Company_Status['Bought_Quantity'])+(price*quantity))/(quantity + Company_Status['Bought_Quantity'])
        Company_Status['Bought_Quantity'] += quantity
        Company_Status['Investment'] = Company_Status['Bought_Price']*Company_Status['Bought_Quantity']
        Company_Status['Current_Price'] = Close
        Company_Status['Current_Valuation'] = Close*Company_Status['Bought_Quantity']
        Company_Status['Status'] = (Company_Status['Current_Valuation'] - Company_Status['Investment'])/Company_Status['Investment']
        portfolio['invested_money'] += price*quantity
        portfolio['Current_funds'] += (Close * quantity)

    else:
        if Company_Status['Bought_Quantity'] > 0:
            portfolio['Profit/Loss'] += (Company_Status['Current_Valuation']-Company_Status['Investment'])
            Company_Status['Bought_Price'] = 0
            Company_Status['Investment'] = 0
            Company_Status['Current_Valuation'] = 0
            Company_Status['Status'] = 0
            portfolio['invested_money'] -= Open * Company_Status['Bought_Quantity']
            portfolio['Current_funds'] -= (Open * Company_Status['Bought_Quantity'])
            Company_Status['Bought_Quantity'] = 0            
                        
        Company_Status['Sold_Price'] = Open
        Company_Status['Sold_Quantity'] = -1*quantity
        Company_Status['Short_Gains'] = (Open-Close)*(-1*quantity)
        shorts +=  Company_Status['Short_Gains']
        portfolio['Profit/Loss'] += Company_Status['Short_Gains']
    
    
    display(ticker, portfolio['Stocks'][ticker],portfolio['invested_money'],portfolio['Current_funds'])
    print()
    

'WABCOINDIA'

{'Bought_Price': 6914.475,
 'Current_Price': 6874.8,
 'Bought_Quantity': 4,
 'Sold_Price': 6906.2,
 'Sold_Quantity': 0,
 'Investment': 27657.9,
 'Current_Valuation': 27499.2,
 'Status': -0.005737962752052785,
 'Short_Gains': -0.0}

2306015.000000001

2284360.4000000004

'DEEPAKNTR'

...

3197080.250000003

'HIMATSEIDE'

{'Bought_Price': 0,
 'Current_Price': 215.45,
 'Bought_Quantity': 0,
 'Sold_Price': 214.3,
 'Sold_Quantity': 0,
 'Investment': 0,
 'Current_Valuation': 0,
 'Status': 0,
 'Short_Gains': -0.0}

2909460.1000000006

3187265.300000003

In [455]:
portfolio['Net_Status'] = ((portfolio['Current_funds'] -portfolio['invested_money'])/portfolio['invested_money'])*100
print(portfolio['invested_money'],portfolio['Current_funds'],portfolio['Net_Status'] )

2909460.1000000006 3187265.300000003 9.548341975887638


In [459]:
print(shorts, portfolio['Profit/Loss'])

-2236.2499999999973 276606.575


In [460]:
share = 0
pos_tickers = 0
neg_tickers = 0 
profit = 0
loss = 0
for ticker in portfolio['Stocks']:
    if portfolio['Stocks'][ticker]['Bought_Quantity'] > 0:
        porl = portfolio['Stocks'][ticker]['Current_Valuation']-portfolio['Stocks'][ticker]['Investment']
        print(ticker, porl)
        if porl > 0:
            pos_tickers += 1
            profit += porl
        else:
            neg_tickers += 1
            loss += porl
        share+= 1
print(share)



AIAENG -36.0
APLAPOLLO -4010.8499999999985
AUBANK 59.07500000000255
AJANTPHARM -2282.5
AKZOINDIA -724.1999999999971
ALKEM 51.599999999998545
AMBUJACEM -191.25
ASIANPAINT 38.39999999999782
AUROPHARMA -11.19999999999709
AVANTIFEED 1.2999999999992724
BASF 44.099999999998545
BEML 40.20000000000073
BALKRISIND -14.299999999999272
BALMLAWRIE 13678.799999999996
BATAINDIA -55.35000000000218
BAYERCROP -26.700000000000728
BERGEPAINT 137.95000000000073
BPCL -1862.7999999999993
BOSCHLTD -57.849999999998545
CADILAHC -33.349999999998545
CANFINHOME 8.75
CENTURYPLY -74.09999999999854
CIPLA 69.0
COCHINSHIP 2459.0999999999913
COROMANDEL 2.2999999999992724
CROMPTON -345.9750000000022
CYIENT -9.775000000001455
DCMSHRIRAM 153.0
DEEPAKNTR -11835.700000000004
DIVISLAB -32.20000000000073
EDELWEISS 73456.02500000001
EICHERMOT 192.1999999999971
EMAMILTD 7215.299999999999
ENDURANCE -115.59999999999854
FDC -217.5
FSL 229.90000000000146
GHCL -49.19999999999709
GALAXYSURF 136.34999999999854
GILLETTE 5609.75
GLAXO -3

In [461]:
print(pos_tickers, neg_tickers, profit, loss)

58 64 243681.25 -222841.07499999995


In [76]:
date_today = '2020-07-24'
profitable_companys = 0
loss_companys = 0
profit_earned = 0
loss = 0
today_funds = 0
def update_portfolio(ticker):
    global today_funds, profitable_companys, loss_companys, profit_earned, loss
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)   
    df.index = df.Date
    company_holdings = portfolio['Stocks'][ticker] 
    #print(ticker , company_holdings['Bought_Quantity'], company_holdings['Investment'])
    if str(date_today) in df.index:
        current_valuation = company_holdings['Bought_Quantity']*df.loc[str(date_today)]['Close']
    else:
        print(ticker, "DATA NOT AVAILABLE")
    today_funds += current_valuation
    company_holdings['Current_Valuation'] = current_valuation
    company_holdings['Status'] = ((current_valuation - company_holdings['Investment'])/company_holdings['Investment'])*100
    if company_holdings['Status'] > 0:
        profitable_companys += 1
        profit_earned +=  current_valuation - company_holdings['Investment']
    else:
        loss_companys += 1
        loss += company_holdings['Investment'] - current_valuation
        
    #print("\t\t", company_holdings['Current_Valuation'],company_holdings['Status'] )
    
for ticker in tickers:
    if portfolio['Stocks'][ticker]['Bought_Quantity'] > 0:
        update_portfolio(ticker)

portfolio['Current_funds'] = today_funds
portfolio['Net_Status'] = (portfolio['Current_funds'] - portfolio['invested_money'] )/portfolio['invested_money']
print(portfolio['invested_money'], portfolio['Current_funds'], portfolio['Net_Status'] )

817716.2999999999 815872.6999999997 -0.0022545716650141496


In [77]:
print(portfolio['invested_money'], portfolio['Current_funds'], portfolio['Net_Status'] )
print(profitable_companys, loss_companys, profit_earned, loss)

817716.2999999999 815872.6999999997 -0.0022545716650141496
69 146 5501.974999999993 7345.57499999999


817716.2999999999 818129.4500000003 0.050524858071237244
